In [21]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromMeshVertices
from Eval.Clouds.Sphere import CreateSpherePointCloud

In [28]:
# Create Base Clouds for Comparison, Created Programatically for High Accuracy.

baseSphere = CreateSpherePointCloud(diameter = 1.0, resolution=10000)
baseSphere.paint_uniform_color([0.5, 0.5, 0.5])

PointCloud with 10000 points.

In [19]:
# Get the Obtained Data Clouds for Comparison, on the Base Programatically Created Clouds.

evalDataDir = "../data/point_e/P-E_Sphere.ply"

targetSphere = CreatePointcloudFromMeshVertices(evalDataDir)
targetSphere.paint_uniform_color([1, 0, 0])

Exception: Provided Mesh Directory does not exist.

In [34]:
# Quick Visualization of both the Obtained Cloud and the Base Cloud.

o3d.visualization.draw_geometries_with_editing([baseSphere, targetSphere], width=800, height=600)